##  CAREER STATS

### This program is designed to create a single dataframe compiled of the carrer stats of every NFL skill position player that competed in the 2012 season. The program has two parts, the first is to create a dataframe comprised of the career stats of the 2012 qbs through the 2011 season. The next step is too use the carrer stats as the initial state of a player's moving average for all relevant characteristics from the 2012 through the 2017 NFL seasons

In [4]:
# Import packages
import pandas as pd
import numpy as np

In [5]:
# read in player's statistical data from the 2012 -2017 NFL seasons
df = pd.read_csv('input/qb/dk_qb_stats-Copy1.csv')

In [10]:
# Read in career data of NFL player stats
rushing = pd.read_csv('input/qb/Career_Stats_Rushing-Copy1.csv')
passing= pd.read_csv('input/qb/Career_Stats_Passing-Copy1.csv')
receiving = pd.read_csv('input/qb/Career_Stats_Receiving-Copy1.csv')

In [11]:
""" A Function designed to parse career NFL data into useable form for implemntation into qb_career_stats dataframe"""



def parse_nfl_stats(df):
    df = df.drop('Player Id', axis=1) # Drop player id from dataframe
    df['Name'] = df['Name'].apply(lambda x: x.split(','))# Split name attribute of every observation by the comma 
    df['Name'] = (df['Name'].apply(lambda x: (x[1] + ' ' + x[0]).strip())) # reverse the name to match carrer_stats dataframe
    df.reset_index(inplace=True)
    df.drop('index', axis=1, inplace=True)
    df.rename(columns=lambda x: x.strip())
    return df

receiving = parse_nfl_stats(receiving)
rushing = parse_nfl_stats(rushing)
passing = parse_nfl_stats(passing)





In [12]:
rushing.rename(columns=lambda x: x.strip(), inplace =True)
passing.rename(columns=lambda x: x.strip(), inplace =True)
receiving.rename(columns=lambda x: x.strip(), inplace =True)

In [13]:
drop_rush = ['Rushing Attempts Per Game','Yards Per Carry','Rushing Yards Per Game','Longest Rushing Run',
             'Rushing First Downs','Percentage of Rushing First Downs',
             'Rushing More Than 20 Yards','Rushing More Than 40 Yards']
rushing.drop(drop_rush, axis=1, inplace=True)

In [14]:
drop_pass = ['Passing Yards Per Attempt', 'Completion Percentage','Percentage of TDs per Attempts', 'Pass Attempts Per Game','Passing Yards Per Game','Int Rate','Longest Pass',
       'Passes Longer than 20 Yards', 'Passes Longer than 40 Yards', 'Sacks',
       'Sacked Yards Lost','Passer Rating' ]
passing.drop(drop_pass, axis=1, inplace=True)

In [15]:
drop_rec = ['Yards Per Game','Yards Per Reception','Longest Reception',
                    'Receptions Longer than 20 Yards','Receptions Longer than 40 Yards','First Down Receptions']
receiving.drop(drop_rec, axis =1, inplace=True)

In [16]:
"""A function designed to change data type of every integer attribute to float 32. Neccessary for groupby function"""

def float_32(df):
    strings = ['Name','Position','Team'] # Define columns to be unchanged
    temp = df[strings]  # store string variables in temporary dataframe
    df = df.drop(strings, axis =1) # drop string variables from original dataframe
    df = df.apply(lambda x: pd.to_numeric(x.astype(str).str.replace(',',''), errors='coerce')).astype('float32') # command to repalce all commas in integers so column dtypes can be changed
    df = pd.merge(temp, df, how='outer', left_index=True, right_index=True) # Merge temp df and altered df
    df = df[df['Year'] > 1999]
    df = df[df['Year']< 2012]
    df.reset_index(inplace=True)
    df.drop('index', axis =1 , inplace=True)

    return df

rushing = float_32(rushing)
receiving = float_32(receiving)
passing = float_32(passing)

In [17]:
passing.head()

,Name,Position,Team,Year,Games Played,Passes Attempted,Passes Completed,Passing Yards,TD Passes,Ints
0,Tommy Maddox,NaN,Pittsburgh Steelers,2005.0,4.0,71.0,34.0,406.0,2.0,4.0
1,Tommy Maddox,NaN,Pittsburgh Steelers,2004.0,4.0,60.0,30.0,329.0,1.0,2.0
2,Tommy Maddox,NaN,Pittsburgh Steelers,2003.0,16.0,519.0,298.0,3414.0,18.0,17.0
3,Tommy Maddox,NaN,Pittsburgh Steelers,2002.0,15.0,377.0,234.0,2836.0,20.0,16.0
4,Tommy Maddox,NaN,Pittsburgh Steelers,2001.0,3.0,9.0,7.0,154.0,1.0,1.0


In [23]:
"""The career dataset being used has no value for the position attribute. 
This function defines which players in the career dataset are quarterbacks by matching the palyers inmy quarterbacks
2012 dataset with the players in my career dataset"""

indices = []
for idx,name in enumerate(passing['Name']):
    if name not in players:
        indices.append(idx)

passing.drop(indices, inplace = True)
passing['Position'] = 'QB'
passing.reset_index(inplace=True,drop=True)

In [24]:
temp = rushing.iloc[indices]

temp['Position'] = 'QB'
temp.fillna(0, inplace=True)
temp.reset_index(inplace=True, drop=True)

# Merge two dataframes to get all passing and rushing statistics for the complete career stats of the qb's in the 2012 season

qb = pd.merge(passing, temp, how='outer')
qb.head()

C:\Users\bobal\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\bobal\Anaconda2\envs\py36\lib\site-packages\pandas\core\frame.py:2754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


,Name,Position,Team,Year,Games Played,Passes Attempted,Passes Completed,Passing Yards,TD Passes,Ints,Rushing Attempts,Rushing Yards,Rushing TDs,Fumbles
0,Charlie Batch,QB,Pittsburgh Steelers,2011.0,4.0,24.0,15.0,208.0,0.0,1.0,3.0,-2.0,0.0,0.0
1,Charlie Batch,QB,Pittsburgh Steelers,2010.0,3.0,49.0,29.0,352.0,3.0,3.0,7.0,30.0,0.0,2.0
2,Charlie Batch,QB,Pittsburgh Steelers,2009.0,1.0,2.0,1.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Charlie Batch,QB,Pittsburgh Steelers,2007.0,7.0,36.0,17.0,232.0,2.0,3.0,12.0,-7.0,0.0,0.0
4,Charlie Batch,QB,Pittsburgh Steelers,2006.0,8.0,53.0,31.0,492.0,5.0,0.0,13.0,15.0,0.0,1.0


In [29]:
career_qb = qb.groupby('Name').sum()

In [31]:
career_qb.index

Index(['Aaron Rodgers', 'Adrian Peterson', 'Ahmard Hall', 'Al Del Greco',
       'Aldrick Robinson', 'Alex Smith', 'Amp Lee', 'Andy Dalton',
       'Anquan Boldin', 'Anthony Allen',
       ...
       'Tyrone Calico', 'Tyrone Wheatley', 'Tyson Alualu', 'Vonta Leach',
       'William Floyd', 'Willie Jackson', 'Woody Dantzler', 'Xavier Omon',
       'Yamon Figurs', 'Zach Miller'],
      dtype='object', name='Name', length=356)

In [25]:
df.head()

,Player,Team,Week,Opp,Comp,Att,Pct,Yds,Yds/Att,TD,Int,ru_att,ru_yds,yds/ru_att,ru_td,points,year,def_ru_rk,def_pass_rk
0,Matt Ryan,ATL,1,KC,23,31,74.0,299,10.0,3,0,3,25,8.0,1,32.46,2012,25,25
1,Robert Griffin III,WAS,1,NO,19,26,73.0,320,12.0,2,0,9,42,5.0,0,25.00,2012,31,31
2,Tony Romo,DAL,1,NYG,22,29,76.0,307,11.0,3,1,5,12,2.0,0,23.48,2012,12,12
3,Aaron Rodgers,GB,1,SF,30,44,68.0,303,7.0,2,1,5,27,5.0,0,22.82,2012,2,2
4,Drew Brees,NO,1,WAS,24,52,46.0,339,7.0,3,2,0,0,0.0,0,21.56,2012,22,22


In [26]:
temp = df.groupby(['year','Player','Week']).sum()
temp.head()

Comp  Att   Pct  Yds  Yds/Att  TD  Int  ru_att  \
year Player        Week                                                   
2012 Aaron Rodgers 1       30   44  68.0  303      7.0   2    1       5   
                   2       22   32  69.0  219      7.0   1    1       3   
                   3       26   39  67.0  223      6.0   0    0       2   
                   4       31   41  76.0  319      8.0   4    1       5   
                   5       20   32  63.0  235      7.0   3    1       5   

                         ru_yds  yds/ru_att  ru_td  points  def_ru_rk  \
year Player        Week                                                 
2012 Aaron Rodgers 1         27         5.0      0   22.82          2   
                   2         -6        -2.0      0   10.16          3   
                   3         17         9.0      0   10.62          1   
                   4         13         3.0      0   28.06         31   
                   5         57        11.0      0   25.10         21   

                         def_pass_rk  
year Player        Week               
2012 Aaron Rodgers 1               2  
                   2               3  
                   3               1  
                   4              31  
                   5              21

In [6]:
df_2012= df[df['year']==2012]
#df_2012.index = df_2012['Player']

In [120]:
for idx, player in enumerate(df_2012['Player']):
    if player not in name:
        print(player)

In [230]:
name = df_2012['Player'].unique()
name

array(['Matt Ryan', 'Robert Griffin III', 'Tony Romo', 'Aaron Rodgers',
       'Drew Brees', 'Mark Sanchez', 'Peyton Manning', 'Joe Flacco',
       'Blaine Gabbert', 'Jay Cutler', 'Carson Palmer', 'Alex Smith',
       'Tom Brady', 'Matt Cassel', 'Ben Roethlisberger', 'Michael Vick',
       'Ryan Fitzpatrick', 'Matt Schaub', 'Philip Rivers', 'Eli Manning',
       'Cam Newton', 'Matthew Stafford', 'Sam Bradford', 'Josh Freeman',
       'Jake Locker', 'Andrew Luck', 'Christian Ponder', 'Russell Wilson',
       'Kevin Kolb', 'Andy Dalton', 'John Skelton', 'Ryan Tannehill',
       'Tyrod Taylor', 'Matt Hasselbeck', 'Colin Kaepernick', 'Tim Tebow',
       'Chase Daniel', 'Bruce Gradkowski', 'Brandon Weeden', 'Chad Henne',
       'Shaun Hill', 'Derek Anderson', 'David Carr', 'Luke McCown',
       'TJ Yates', 'Kyle Orton', 'Brock Osweiler', 'Graham Harrell',
       'Kirk Cousins', 'Rusty Smith', 'Brady Quinn', 'Jason Campbell',
       'Joe Webb', 'Matt Moore', 'Ryan Mallett', 'Kellen Clemens',

In [232]:
for idx,player in enumerate(df_2012['Player']):
    if player in list(name):
        #print(player)
        df.iloc[idx]['Games Played'] = 0
    #df_2012['Player'].map(lambda x: career_qb.loc[x]['Games Played'] + 1)
      #  name = name[name !=player]
    #else:
     #   raw_qb_2012['Games Played'] = raw_qb_2012['Games Played'] +1
        

C:\Users\bobal\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\bobal\Anaconda2\envs\py36\lib\site-packages\pandas\core\series.py:747: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc[key] = value


In [234]:
df_2012.columns

Index(['Player', 'Team', 'Week', 'Opp', 'Comp', 'Att', 'Pct', 'Yds', 'Yds/Att',
       'TD', 'Int', 'ru_att', 'ru_yds', 'yds/ru_att', 'ru_td', 'points',
       'year', 'def_ru_rk', 'def_pass_rk'],
      dtype='object')

In [240]:
raw_qb_2012['Games Played'] = raw_qb_2012['Player'].map(lambda x: career_qb.loc[x]['Games Played'])

In [242]:
df_2012 = raw_qb_2012

In [243]:
df_2012.head()

,Player,Team,Week,Opp,Comp,Att,Pct,Yds,Yds/Att,TD,Int,ru_att,ru_yds,yds/ru_att,ru_td,points,year,def_ru_rk,def_pass_rk,Games Played
0,Matt Ryan,ATL,1,KC,23,31,74.0,299,10.0,3,0,3,25,8.0,1,32.46,2012,25,25,62.0
1,Robert Griffin III,WAS,1,NO,19,26,73.0,320,12.0,2,0,9,42,5.0,0,25.00,2012,31,31,0.0
2,Tony Romo,DAL,1,NYG,22,29,76.0,307,11.0,3,1,5,12,2.0,0,23.48,2012,12,12,105.0
3,Aaron Rodgers,GB,1,SF,30,44,68.0,303,7.0,2,1,5,27,5.0,0,22.82,2012,2,2,69.0
4,Drew Brees,NO,1,WAS,24,52,46.0,339,7.0,3,2,0,0,0.0,0,21.56,2012,22,22,154.0


In [46]:
for idx,week in enumerate(df_2012['Week']):
    if week ==1:
        df_2012.iloc[idx]['Games Played'] = df_2012['Player'].map(lambda x: career_qb.loc[x]['Games Played'])
    #else:
     #   df_2012.iloc[idx]['Games Played'] = df_2012.iloc[idx]['Games Played'] + 1            
        

C:\Users\bobal\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\bobal\Anaconda2\envs\py36\lib\site-packages\pandas\core\series.py:747: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc[key] = value


In [47]:
df_2012.head()

,Player,Team,Week,Opp,Comp,Att,Pct,Yds,Yds/Att,TD,Int,ru_att,ru_yds,yds/ru_att,ru_td,points,year,def_ru_rk,def_pass_rk
0,Matt Ryan,ATL,1,KC,23,31,74.0,299,10.0,3,0,3,25,8.0,1,32.46,2012,25,25
1,Robert Griffin III,WAS,1,NO,19,26,73.0,320,12.0,2,0,9,42,5.0,0,25.00,2012,31,31
2,Tony Romo,DAL,1,NYG,22,29,76.0,307,11.0,3,1,5,12,2.0,0,23.48,2012,12,12
3,Aaron Rodgers,GB,1,SF,30,44,68.0,303,7.0,2,1,5,27,5.0,0,22.82,2012,2,2
4,Drew Brees,NO,1,WAS,24,52,46.0,339,7.0,3,2,0,0,0.0,0,21.56,2012,22,22


In [39]:
career_qb.drop('Year',axis=1, inplace=True)

In [544]:
for player in raw_qb_2012['Player']:
    for col in raw_qb_2012[raw_qb_2012['Player']==player].columns:
        if raw_qb_2012[raw_qb_2012['Player']==player][col].dtypes != 'object':
            np.mean(raw_qb_2012[raw_qb_2012['Player']==player][col])
    

In [40]:
career_qb[career_qb.index == 'Matt Ryan']['Games Played']

Name
Matt Ryan    62.0
Name: Games Played, dtype: float64

In [391]:
df['Games_Played'] = career_qb['Games Played'] + 1

In [22]:
players = df_2012['Player'].unique() # Define the all the QB's for the 2012 2017 NFL Season

In [48]:
# A temporary dataframe used to store qb rushing career rushing stats
indices = []
for idx,name in enumerate(rushing['Name']):
    if name in players:
        indices.append(idx)

In [63]:
# add all stats of of a players career for each player and groupby player
career_qb = qb.groupby('Name').sum()
career_qb.drop('Year', axis=1, inplace=True)

In [64]:
career_qb.head()

,Games Played,Passes Attempted,Passes Completed,Passing Yards,TD Passes,Ints,Rushing Attempts,Rushing Yards,Rushing TDs,Fumbles
Name,,,,,,,,,,
Aaron Rodgers,69.0,2113.0,1381.0,17366.0,132.0,38.0,249.0,1183.0,16.0,8.0
Alex Smith,70.0,1959.0,1137.0,12543.0,68.0,58.0,181.0,629.0,4.0,8.0
Andy Dalton,16.0,516.0,300.0,3398.0,20.0,13.0,37.0,152.0,1.0,1.0
Ben Roethlisberger,114.0,3313.0,2090.0,26579.0,165.0,100.0,293.0,944.0,14.0,16.0
Blaine Gabbert,15.0,413.0,210.0,2214.0,12.0,11.0,48.0,98.0,0.0,8.0


In [370]:
career_qb.isnull().sum()

Games Played        0
Passes Attempted    2
Passes Completed    2
Passing Yards       2
TD Passes           2
Ints                2
Rushing Attempts    0
Rushing Yards       0
Rushing TDs         0
Fumbles             0
dtype: int64

In [59]:
career_qb = career_qb.reset_index()
career_qb['Name'] = career_qb['Name'].apply(lambda x: x.strip())

KeyError: '[  1.05000000e+02   2.59200000e+03   1.67200000e+03   2.08340000e+04\n   1.49000000e+02   7.20000000e+01   1.58000000e+02   4.59000000e+02\n   4.00000000e+00   3.20000000e+01] not in index'

In [42]:
# Manually enter in career stats for missing players from career dataframe

career_qb.loc['Tony Romo'] = [105,2592,1672,20834,149,72,158,459,4,32]
career_qb.loc['Michael Vick'] = [111,2538,1422,17912,111,82,729,5219,33,59]
career_qb.loc['Matt Schaub'] = [102,2279,1466,17936,98,58,157,337,4,25]
career_qb.loc['Josh Freeman'] = [31,1315,795,9257,51,46,153,238,4,14]
career_qb.loc['John Skelton'] = [13,401, 211,2575,13,16,11,38,0,4]
career_qb.loc['Bruce Gradkowski'] = [39,698,370,3992, 21,26,82,333,0,13]
career_qb.loc['TJ Yates'] = [6,134,82,949,3,3,14,57,0,4]
career_qb.loc['Rusty Smith'] = [2,40,20,200,0,4,0,0,0,0]

In [43]:
Rookies = ['Robert Griffin III','Jake Locker','Andrew Luck', 'Russell Wilson',
           'Ryan Tannehill','Brandon Weeden','Brock Osweiler','Kirk Cousins','Nick Foles','Ryan Lindley',
          'Greg McElroy', 'Thaddeus Lewis']

In [44]:
for rook in Rookies:
    career_qb.loc[rook] = 0

In [72]:
career_rb.reset_index(inplace=True)
career_rb['Name'] = career_rb['Name'].apply(lambda x: x.strip())

In [82]:
temp = career_rb.iloc[indices]

In [5]:
raw_qb_2012 = raw_qb_2012.rename(columns={'Att.1':'rush_att',
                       'Yds.1':'rush_yds',
                       'Yds/Att.1':'rush_yd/att',
                       'TD.1':'rush_td'})

In [6]:
team_stats = raw_qb_2012.groupby(['Player','Week','Opp']).sum()

In [7]:
team_stats.drop('Fantasy Points', axis=1, inplace=True)

In [8]:
team_stats.head(), team_stats.shape

(                        Comp  Att  Pct  Yds  Yds/Att  TD  Int  QB Rating  \
 Player        Week Opp                                                     
 Aaron Rodgers 1    SF     30   44   68  303        7   2    1         93   
               2    CHI    22   32   69  219        7   1    1         85   
               3    SEA    26   39   67  223        6   0    0         81   
               4    NO     31   41   76  319        8   4    1        120   
               5    IND    20   32   63  235        7   3    1        103   
 
                         rush_att  rush_yds  rush_yd/att  rush_td  
 Player        Week Opp                                            
 Aaron Rodgers 1    SF          5        27            5        0  
               2    CHI         3        -6           -2        0  
               3    SEA         2        17            9        0  
               4    NO          5        13            3        0  
               5    IND         5        57        

In [9]:
# Define a dataframe to contain the career averages for all 2012 QBs
def create_career_dataframe(df):
    columns = []
    for col in df.columns:
        columns.append('{}_car'.format(col.lower()))
    new_df = pd.DataFrame(columns=columns, index =df.index)
    return new_df

In [10]:
qb_career_2012 = create_career_dataframe(team_stats)

In [11]:
# Week 1 stats are player's career average through 2011 season
# Populate current dataframe with current year averages of players
for col in team_stats:  # Iterate through every attribute in the team_stats dataframe
    points=[
        
    ]  # Create an empty list to hold the season average of the current attribute
    for player in team_stats.index.levels[0]:  # Iterate through every player in the team_stats df
        total =[] # create an empty list to hold the weekly attributes value
        for week in team_stats.loc[player].index: # Iterate throough every week for the current player
            total.append(team_stats.loc[player].loc[week][col]) # Add player's value for the current week's attribute to the total list
            points.append(np.mean(total))  # Add the average of the total list to the season average list
    qb_career_2012['{}_car'.format(col.lower())] = points # Insert season average list for current attribute into the current stats dataframe
    

In [12]:
qb_career_2012.shape

(629, 12)

In [13]:
qb_career_2012.to_csv('input/2012/qb/career_qb_stats.csv', index = True)

In [27]:
qb_career_2012['score'] = team_stats['Fantasy Points']

In [28]:
qb_career_2012.head()

comp_car    att_car  pct_car     yds_car  yds/att_car  \
Player        Week Opp                                                          
Aaron Rodgers 1    SF      30.00  44.000000     68.0  303.000000     7.000000   
              2    CHI     26.00  38.000000     68.5  261.000000     7.000000   
              3    SEA     26.00  38.333333     68.0  248.333333     6.666667   
              4    NO      27.25  39.000000     70.0  266.000000     7.000000   
              5    IND     25.80  37.600000     68.6  259.800000     7.000000   

                        td_car   int_car  qb rating_car  rush_att_car  \
Player        Week Opp                                                  
Aaron Rodgers 1    SF     2.00  1.000000      93.000000      5.000000   
              2    CHI    1.50  1.000000      89.000000      4.000000   
              3    SEA    1.00  0.666667      86.333333      3.333333   
              4    NO     1.75  0.750000      94.750000      3.750000   
              5    IND    2.00  0.800000      96.400000      4.000000   

                        rush_yds_car  rush_yd/att_car  rush_td_car  \
Player        Week Opp                                               
Aaron Rodgers 1    SF      27.000000             5.00          0.0   
              2    CHI     10.500000             1.50          0.0   
              3    SEA     12.666667             4.00          0.0   
              4    NO      12.750000             3.75          0.0   
              5    IND     21.600000             5.20          0.0   

                        fantasy points_car  score  
Player        Week Opp                             
Aaron Rodgers 1    SF            22.820000  22.82  
              2    CHI           16.490000  10.16  
              3    SEA           14.533333  10.62  
              4    NO            17.915000  28.06  
              5    IND           19.352000  25.10